# Predicting Demonstration Size on Demographic Characteristics and Salience

## GW DATS 6103: Introduction to Data Mining Final Project
### Name: Alexander D. Silberman

Note: Data is limited to 2017–2019, as COVID may substantially throw off the data.

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

from census import Census
from us import states

import censusdis.data as ced
from censusdis.datasets import ACS1_PROFILE
from censusdis import states

## Data Acquisition and Wrangling

### Results Dataset: Crowd Counting Consortium Phase 1 (2017-2020)

Our results dataset: Crowd Counting Consortium Phase 1 (2017-2020): https://ash.harvard.edu/programs/crowd-counting-consortium/

In [3]:
crowd_data = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")
crowd_data.head(5)

C:\Users\alexa\AppData\Local\Temp\ipykernel_41792\206043875.py:1: DtypeWarning: Columns (6,22,31,32,33,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  crowd_data = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")


,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,source_28,source_29,source_30,notes,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code
0,2017-01-01,Washington,DC,Lafayette Square Park,0.0,vigil,NaN,NaN,"for banning nuclear weapons, for peace",0.0,...,NaN,NaN,NaN,White House Peace Vigil continuous since June ...,38.907192,-77.036871,Washington,District of Columbia,DC,11001.0
1,2017-01-01,Mankato,MN,NaN,0.0,vigil,NaN,Peace Vigil Mankato,for peace,0.0,...,NaN,NaN,NaN,every Sunday since 2001,44.163578,-93.999400,Mankato,Blue Earth County,MN,27013.0
2,2017-01-01,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,general protestors,"against the Dakota Access Pipeline, for indige...",1.0,...,NaN,NaN,NaN,hung banner from stadium roof during NFL game,44.977753,-93.265011,Minneapolis,Hennepin County,MN,27053.0
3,2017-01-01,Little Compton,RI,Town Green,0.0,vigil,NaN,Sakonnet Peace Alliance,"for peace, for gun control, for climate action",1.0,...,NaN,NaN,NaN,every Sunday since 2003,41.510103,-71.171156,Little Compton,Newport County,RI,44005.0
4,2017-01-01,Oak Ridge,TN,Y-12 National Security Complex,0.0,vigil,NaN,Oak Ridge Environmental Peace Alliance,for abolishing nuclear weapons,0.0,...,NaN,NaN,NaN,every Sunday since the late 1990s,36.010356,-84.269645,Oak Ridge,Anderson County,TN,47001.0


In [4]:
# source_cols = np.array(["source_"]*30) + np.arange(1,31).astype(str)
# crowd_data.drop(columns=source_cols, inplace=True)
# crowd_data.head(5)


### American Community Survey

In [50]:
api_key = "29e7dfea2f8b253a0a10ccd9626f78e49f4f0a4f"

def regex_filter(string, myregex):
    if string:
        mo = re.search(myregex, string)
        if mo:
            return True
        else:
            return False
    else:
        return False

census_vars = ced.variables.search(ACS1_PROFILE, 2017)
census_vars = census_vars[census_vars["GROUP"] != "N/A"]
census_vars_pcts = census_vars[census_vars["VARIABLE"].apply(regex_filter,myregex=r"PE$")]
census_vars_pcts

,YEAR,DATASET,GROUP,VARIABLE,LABEL,SUGGESTED_WEIGHT,VALUES
11,2017,acs/acs1/profile,DP02PR,DP02PR_0001PE,Percent!!HOUSEHOLDS BY TYPE!!Total households,NaN,None
13,2017,acs/acs1/profile,DP02PR,DP02PR_0002PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
15,2017,acs/acs1/profile,DP02PR,DP02PR_0003PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
17,2017,acs/acs1/profile,DP02PR,DP02PR_0004PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
19,2017,acs/acs1/profile,DP02PR,DP02PR_0005PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
...,...,...,...,...,...,...,...
1347,2017,acs/acs1/profile,DP05,DP05_0085PE,Percent!!HISPANIC OR LATINO AND RACE!!Total po...,NaN,None
1349,2017,acs/acs1/profile,DP05,DP05_0086PE,Percent!!Total housing units,NaN,None
1351,2017,acs/acs1/profile,DP05,DP05_0087PE,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...",NaN,None
1353,2017,acs/acs1/profile,DP05,DP05_0088PE,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...",NaN,None


Get rid of all estimates, keeping only percents. The list of variables includes what should be the total population as a percent—"DP05_0001P"—but, as that would always be 100%, the wise people at the US Census Bureau made such equal to "DP05_0001E", the total population estimate, necessitating no further adjustment.

In [51]:
census_data = [ced.download(ACS1_PROFILE, year, pd.concat([pd.Series(["NAME"]), census_vars_pcts["VARIABLE"]], ignore_index=True), 
                           metropolitan_statistical_area_micropolitan_statistical_area ="*", 
                           api_key=api_key,
                           ).set_index("METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STATISTICAL_AREA")# .rename(columns=census_vars)
               for year in range(2017,2020)]


census_data = pd.concat(census_data, keys = np.arange(2017,2020))
census_data

NAME  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                                                 
2017 44220                                                                 Springfield, OH Metro Area   
     44300                                                               State College, PA Metro Area   
     44340                                                                  Statesboro, GA Micro Area   
     44420                                                         Staunton-Waynesboro, VA Metro Area   
     44620                                                               Stevens Point, WI Micro Area   
...                                                                                               ...   
2019 49620                                                                York-Hanover, PA Metro Area   
     49660                                               Youngstown-Warren-Boardman, OH-PA Metro Area   
     49700                                                                   Yuba City, CA Metro Area   
     49740                                                                        Yuma, AZ Metro Area   
     49780                                                                  Zanesville, OH Micro Area   

                                                         DP02PR_0001PE  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2019 49620                                                         NaN   
     49660                                                         NaN   
     49700                                                         NaN   
     49740                                                         NaN   
     49780                                                         NaN   

                                                         DP02PR_0002PE  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2019 49620                                                         NaN   
     49660                                                         NaN   
     49700                                                         NaN   
     49740                                                         NaN   
     49780                                                         NaN   

                                                         DP02PR_0003PE  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2019 49620                                                         NaN   
     49660                                                         NaN   
     49700                                                       

### Most Important Problem Dataset, Second Release (2024)

The aggregated data may be found here: https://williamslaro.github.io/talks/dataset2. Previously, the individual data was used, necessitating much preprocessing.

In [14]:
mipd_cols = ["studyid",
             "entry",
             "fw_start",
             "fw_end",
             "sample",
             "oversample",
             "weightavailable",
             "short",
             "id",
             "weight",
             "mipcode1_r1",
             "mipsubcode1_r1",
             "mipareacode1_r1",
             "mipcode2_r1",
             "mipsubcode2_r1",
             "mipareacode2_r1",
             "mipcode3_r1",
             "mipsubcode3_r1",
             "mipareacode3_r1",
             "mipcode4_r1",
             "mipsubcode4_r1",
             "mipareacode4_r1",
             "mipcode5_r1",
             "mipsubcode5_r1",
             "mipareacode5_r1",
             "mipcode6_r1",
             "mipsubcode6_r1",
             "mipareacode6_r1",
             "mipcode7_r1",
             "mipsubcode7_r1",
             "mipareacode7_r1",
             "mipcode8_r1",
             "mipsubcode8_r1",
             "mipareacode8_r1",
             "mipcode9_r1",
             "mipsubcode9_r1",
             "mipareacode9_r1",
             "mipcode10_r1",
             "mipsubcode10_r1",
             "mipareacode10_r1"]

issue_data_full = pd.DataFrame(columns=mipd_cols)

with pd.read_stata("C:\\Users\\alexa\\Code\\GW DATS\\6103 12 Intro to Data Mining\\Final Project\data\\2024-MIPD\\USMISC2024-MIPD_ind.dta", 
                   chunksize=10000, columns=mipd_cols, convert_categoricals=False) as issue_reader:
    for chunk in issue_reader:
        issue_data_full = pd.concat([issue_data_full, chunk],ignore_index=True)


C:\Users\alexa\AppData\Local\Temp\ipykernel_41792\2038805593.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  issue_data_full = pd.concat([issue_data_full, chunk],ignore_index=True)
C:\Users\alexa\AppData\Local\Temp\ipykernel_41792\2038805593.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  issue_data_full = pd.concat([issue_data_full, chunk],ignore_index=True)
C:\Users\alexa\AppData\Local\Temp\ipykernel_41792\2038805593.py:47: FutureWarning: The behavior of DataFrame concatenation with 

In [11]:
len(issue_data_full)

1009177

There are 1009177 observations in this dataset. These will be limited to one year prior to a given demonstration. As our earliest data is for 2017, no surveys taking place before 2016-01-01 shall be considered.

In [15]:
issue_data_full.describe()

,entry,fw_start,fw_end,weight,mipcode1_r1,mipsubcode1_r1,mipareacode1_r1,mipcode2_r1,mipsubcode2_r1,mipareacode2_r1,...,mipareacode7_r1,mipcode8_r1,mipsubcode8_r1,mipareacode8_r1,mipcode9_r1,mipsubcode9_r1,mipareacode9_r1,mipcode10_r1,mipsubcode10_r1,mipareacode10_r1
count,78404.000000,1009177,1009177,8.648450e+05,1.006261e+06,1.152550e+05,65851.000000,531703.000000,1.055560e+05,21058.000000,...,1853.000000,14433.000000,3.334000e+03,1271.000000,7304.000000,1.509000e+03,300.000000,3786.000000,8.660000e+02,166.000000
mean,1.335008,1987-08-20 04:09:13.567907200,1987-08-25 17:48:23.891388800,4.662456e+02,5.424049e+02,6.127799e+05,646.973364,516.123197,5.957032e+05,533.315415,...,594.299514,414.852352,3.941057e+05,474.896932,494.888965,4.927444e+05,742.756667,380.215003,4.507806e+05,1011.421687
min,1.000000,1939-01-09 00:00:00,1939-01-14 00:00:00,0.000000e+00,1.010000e+02,1.030010e+05,40.000000,101.000000,1.030010e+05,40.000000,...,40.000000,101.000000,1.030010e+05,40.000000,101.000000,1.030010e+05,40.000000,101.000000,1.030010e+05,40.000000
25%,1.000000,1972-09-01 00:00:00,1972-09-25 00:00:00,1.000000e+00,1.030000e+02,4.070020e+05,365.000000,109.000000,4.050010e+05,365.000000,...,365.000000,108.000000,2.030060e+05,365.000000,207.000000,3.090010e+05,365.000000,101.000000,3.080010e+05,365.000000
50%,1.000000,1991-10-15 00:00:00,1991-10-20 00:00:00,2.000000e+00,5.030000e+02,6.040050e+05,645.000000,503.000000,6.040010e+05,365.000000,...,630.000000,407.000000,2.170030e+05,365.000000,601.000000,5.050010e+05,365.000000,308.000000,5.050010e+05,720.000000
75%,1.000000,2005-09-06 00:00:00,2005-09-13 00:00:00,2.160000e+02,8.010000e+02,1.010001e+06,816.000000,610.000000,8.050020e+05,645.000000,...,630.000000,601.000000,5.060000e+05,365.000000,606.000000,6.051390e+05,1600.000000,601.000000,5.060000e+05,1800.000000
max,4.000000,2018-06-05 00:00:00,2018-06-12 00:00:00,1.042350e+06,1.504000e+03,1.201004e+06,1800.000000,1504.000000,1.201001e+06,1800.000000,...,1800.000000,1502.000000,1.201001e+06,1800.000000,1502.000000,1.201002e+06,1800.000000,1502.000000,1.201001e+06,1800.000000
std,0.848412,NaN,NaN,7.509439e+03,4.455091e+02,3.006765e+05,282.568638,401.415764,2.946908e+05,343.012451,...,374.747190,314.350104,2.746691e+05,343.537887,313.241780,2.457078e+05,652.537956,327.941287,2.361090e+05,721.759007


In [12]:
issue_data = issue_data_full[issue_data_full["fw_end"] > np.datetime64("2016-01-01")]
del(issue_data_full)
len(issue_data)

10725

With this limitation, there are only 10725 observations to deal with.

In [13]:
issue_data.describe()

,entry,fw_start,fw_end,weight,mipcode1_r1,mipsubcode1_r1,mipareacode1_r1,mipcode2_r1,mipsubcode2_r1,mipareacode2_r1,...,mipareacode7_r1,mipcode8_r1,mipsubcode8_r1,mipareacode8_r1,mipcode9_r1,mipsubcode9_r1,mipareacode9_r1,mipcode10_r1,mipsubcode10_r1,mipareacode10_r1
count,3504.0,10725,10725,10725.000000,10688.000000,2.061000e+03,136.000000,6107.000000,1372.000,0.0,...,0.0,54.000000,4.400000e+01,0.0,10.0,10.0,0.0,0.0,0.0,0.0
mean,1.0,2017-02-12 20:57:23.916083712,2017-02-17 09:21:54.125874432,1.290072,719.463323,7.225414e+05,826.264706,540.940560,644364.000,NaN,...,NaN,898.037037,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
min,1.0,2016-01-07 00:00:00,2016-01-10 00:00:00,0.059000,101.000000,1.030010e+05,365.000000,102.000000,103001.000,NaN,...,NaN,401.000000,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
25%,1.0,2016-09-27 00:00:00,2016-09-29 00:00:00,0.587500,208.000000,5.040010e+05,686.250000,204.000000,403001.000,NaN,...,NaN,1011.000000,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
50%,1.0,2017-01-04 00:00:00,2017-01-09 00:00:00,0.956882,602.000000,6.080010e+05,731.000000,506.000000,604005.000,NaN,...,NaN,1011.000000,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
75%,1.0,2018-01-14 00:00:00,2018-01-18 00:00:00,1.552900,1012.000000,1.011007e+06,731.000000,901.000000,1010004.000,NaN,...,NaN,1011.000000,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
max,1.0,2018-06-05 00:00:00,2018-06-12 00:00:00,7.612903,1504.000000,1.012045e+06,1600.000000,1504.000000,1012045.000,NaN,...,NaN,1011.000000,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
std,0.0,NaN,NaN,1.158380,500.999539,3.052793e+05,421.284847,379.682161,312522.625,NaN,...,NaN,239.178071,6.322257e-01,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [34]:
issue_data = pd.read_csv("C:\\Users\\alexa\\Code\\GW DATS\\6103 12 Intro to Data Mining\\Final Project\\data\\MIPD-2024-Aggregate\\MIPDV2-All-year.csv",encoding="ANSI",
                         index_col=["year","problem"])
issue_data.head(20)

cat  numsurveys  numrespondents       perc         catname  \
year problem                                                               
1939 1          1           3            8989  45.401001  MIPD (General)   
     2          1           3            8989   6.861000  MIPD (General)   
     3          1           3            8989   0.200000  MIPD (General)   
     4          1           3            8989   0.554000  MIPD (General)   
     5          1           3            8989   4.629000  MIPD (General)   
     6          1           3            8989  33.779999  MIPD (General)   
     7          1           3            8989   0.056000  MIPD (General)   
     8          1           3            8989   0.044000  MIPD (General)   
     9          1           3            8989   1.432000  MIPD (General)   
     10         1           3            8989   4.123000  MIPD (General)   
     11         1           3            8989   0.700000  MIPD (General)   
     12         1           3            8989   0.168000  MIPD (General)   
     13         1           3            8989   1.461000  MIPD (General)   
     14         1           3            8989   0.078000  MIPD (General)   
     15         1           3            8989   0.512000  MIPD (General)   
1940 1          1           1            2907  24.200001  MIPD (General)   
     2          1           1            2907   3.474000  MIPD (General)   
     3          1           1            2907   0.069000  MIPD (General)   
     4          1           1            2907   0.172000  MIPD (General)   
     5          1           1            2907  13.152000  MIPD (General)   

                                             catdesc  \
year problem                                           
1939 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   
     6        general problem categories in the MIPD   
     7        general problem categories in the MIPD   
     8        general problem categories in the MIPD   
     9        general problem categories in the MIPD   
     10       general problem categories in the MIPD   
     11       general problem categories in the MIPD   
     12       general problem categories in the MIPD   
     13       general problem categories in the MIPD   
     14       general problem categories in the MIPD   
     15       general problem categories in the MIPD   
1940 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   

                                   problemname  \
year problem                                     
1939 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   
     6                          Foreign Policy   
     7        International Economic Relations   
     8                             Environment   
     9                                Morality   
     10                               Politics   
     11                           Youth Issues   
     12                                 Groups   
     13                          Other and All   
     14                                    NaN   
     15                     Don't Know/Refused   
1940 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   

       

Limit to relevant years

In [35]:
issue_data = issue_data.loc[2020]
issue_data

KeyError: 2020